In [ ]:
import os
import praw

In [ ]:
from dotenv import load_dotenv
your_reddit_username = "<enter your username here>"
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

In [ ]:
user_agent = f"Scraper 1.0 by /u/{your_reddit_username}"
reddit = praw.Reddit (
client_id=client_id,
client_secret=client_secret,
user_agent=user_agent
)

In [ ]:
from nltk.corpus import wordnet as wn
from word_forms.word_forms import get_word_forms
# "public", "infrastructure", "planning", "urban", "civil",
keyword_list = {"transport", "traffic", "travel", "transit", "experience", "commute", "urban", "civil", "accelerate", "rule", "etiquette", "confuse", "empathy", "application", 
                "device", "digital", "facility", "design", "captain", "jam", "booking", "crossing", "crowd", "walkway", "linkway", "signal", 
                "route", "path", "destination", "bridge", "flyover", "overhead", "barrier", "stuffy", "spacious", "service", "breakdown", "bunching", "replace", "uncomfortable", 
                "journey", "network", "challenge", "recommendation", "suggestion", "affordable", "connectivity", "available", "condition", "speed", "fast", "slow", "smooth", 
                "stress", "welfare", "frequency", "certainty", "uncertainty", "licence", "COE", "management", "rail", "double decker", "operator", "elevator", 
                "escalator", "equipment", "brake", "disruption", "accident", "incident", "fatality", "dangerous", "roadworks", "construction", "deploy", "manpower", "expansion", 
                "extension", "incentive", "child-friendly", "family", "stroller", "pram", "luggage", "baggage", "baby", "pregnant", "infant", "molest", "harassment", 
                "family-friendly", "signage", "announcement", "visibility", "communication", "display", "aware", "behaviour", "navigation", "advisory", "update", "risk", 
                "warning", "injury", "hazard", "assist", "guide", "concession", "sticker", "board", "toilet", "provision", "accommodate", "information", "real-time", "handicap", 
                "hurry", "emergency", "wheelchair", "user", "peak-hour", "difficulty", "infrastructure", "institution", "authority", "ministry", "shuttle", "SBS", 
                "station", "platform", "technology", "engineering", "upgrade", "concern", "need", "grievance", "pain point", "problem", "solution", "delay", "stop", 
                "rush", "wait", "queue", "interchange", "development", "road", "lane", "street", "highway", "expressway", "causeway", "boulevard", "avenue", "bus", "car", "train", 
                "taxi", "cab", "vehicle", "truck", "bike", "bicycle", "motor", "vehicle", "metro", "MRT", "mass rapid transit", "SMRT", "LRT", "light rail transit", "drive", 
                "walk", "ride", "cycle", "park", "arrival", "departure", "duration", "comfort", "quality", "efficiency", "convenience", "regularity", "reliability", "punctual", 
                "timely", "inclusivity", "accessibility", "disability", "EZ-Link", "TransitLink", "simplygo", "CEPAS", "ERP", "electronic road pricing", "maintain", "improve", 
                "seats", "safety", "LTA", "land transport authority", "ticket", "pedestrian", "passenger", "patron", "commuter", "intersection", "demand", "expectation", "system", 
                "innovation", "governance", "PTC", "public transport council" "congestion", "fare", "fuel", "petrol", "diesel", "gas", "mobility", "distance", "mode", "trip"}
for word in keyword_list.copy():
    word_forms = get_word_forms(word, similarity_threshold=0.8)
    for key in word_forms:
        for form in word_forms[key]:
            keyword_list.add(form)

In [ ]:
keyword_list= ['"' + s + '"' for s in keyword_list]
keyword_list = ', '.join(keyword_list)

In [ ]:
subreddits = reddit.subreddit("singapore+asksingapore+singaporehappenings+drivingsg")

# creating lists for storing scraped data
ids=[]
titles=[]
content=[]
comments=[]

# looping over posts and scraping it
chunk_size = 20
iter = 0
while iter <= len(keyword_list)-chunk_size+1: 
    for submission in subreddits.search(keyword_list[iter:iter+chunk_size], time_filter="all"):
        if submission.upvote_ratio > 0.7 and submission.score > 50 and submission.id not in ids: 
            ids.append(submission.id)
            titles.append(submission.title)
            content.append(submission.selftext[:min(len(submission.selftext), 2000)])
            submission_comments = []
            for comment in submission.comments:
                if isinstance(comment, praw.models.MoreComments):
                    continue
                if comment.score > 10: 
                    submission_comments.append(comment.body[:min(len(comment.body), 2000)])
            comments.append(submission_comments)

    iter = iter + chunk_size

In [ ]:
df = pd.DataFrame() 
df['id'] = ids
df['title'] = titles
df['content'] = content
df['comment'] = comments

df.to_csv('data/reddit_data.csv')
df.to_json('data/reddit_data.json')

In [ ]:
import google.generativeai as genai
import pandas as pd
genai.configure(api_key=<api_key>)

prompt1 = """Your task is to distill the essence of a given reddit post into a concise summary capturing the key points and essential information, 
within a 10-word limit.\n
If the post is not related to the subject of transportation, simply output 'unrelated' without any text formatting.\n
If, however, the post is about transportation, extract the following information.\n
Summary\n
Category 1 - The most appropriate category for the type of post from the following: complaint, suggestion, praise, query, information/news. Do not make up any categories of your own. If you are unsure of the category, put the category as 'other'.\n 
Category 2 - The most appropriate category for the subject of the post from the following: bus service, train service, private transport, walk/cycle, policy/regulation, infrastructure/facilities/systems. Do not make up any categories of your own. If you are unsure of the category, put the category as 'other'.\n
Output the extracted  information in the following format: 'summary: <summary>\ncategory 1: <category 1>\ncategory 2: <category 2>'
"""



safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

model = genai.GenerativeModel("gemini-pro", safety_settings=safety_settings)

def generate_gemini_content(transcript_text, prompt, model):
    response = model.generate_content(prompt + transcript_text)
    return response.text

In [ ]:
summaries = list()
categories_1 = list()
categories_2 = list()
#df = df.iloc[:10].copy(deep=True)
for index, row in df.iterrows():
    #print(index)
    flag = False
    if not len(row["content"]) == 0:
        try: 
            output = generate_gemini_content(row["content"], prompt1, model)
        except:
            #print(f'error at {index}')
            df = df.drop(index)   
        else:
            #print(index)
            flag = True          
    elif not len(row["title"]) == 0:
        try: 
            output = generate_gemini_content(row["title"], prompt1, model)
        except:
            #print(f'error at {index}')
            df = df.drop(index)
        else:
            #print(index)
            flag = True
    if flag:
        if output.lower() == "unrelated":
            df = df.drop(index)
            continue
        output = output.split('\n')
        output = [i for i in output if i != '']
        print(f'{index}: {output}')
        flag = 0
        try:
            flag = 1
            summaries.append(output[0].split(':')[-1].strip())
            flag = 2
            categories_1.append(output[-2].split(':')[-1].strip())
            flag = 3
            categories_2.append(output[-1].split(':')[-1].strip()) 
        except:
            message = "couldn't generate"
            if flag == 1:
                summaries.append(message)
                categories_1.append(message)
                categories_2.append(message)
            if flag == 2:
                categories_1.append(message)
                categories_2.append(message)
            if flag == 3:
                categories_2.append(messgae)
        #categories_3.append(output[-1].split(':')[-1].lstrip())


In [ ]:
df["summary"] = summaries
df["category1"] = categories_1
df["category2"] = categories_2

In [ ]:
categories_3 = list()

cross_reference = {'bus service': ['frequency', 'punctuality', 'accessibility', 'affordability', 'quality', 'safety', 'connectivity', 'crowding', 'infrastructure/facilities'],
                  'train service': ['frequency', 'punctuality', 'accessibility', 'affordability', 'quality', 'safety', 'connectivity', 'crowding', 'infrastructure/facilities'],
                  'private transport': ['cost', 'parking', 'safety', 'traffic/congestion', 'connectivity'],
                  'walk/cycle': ['safety', 'infrastructure/facilities'],
                  'policy/regulation': ['enforcement', 'compliance', 'transparency/communication', 'impact'],
                  'other': 'other'
                  }

prompt2 = f"""Your task is to categorise social media content about transportation in Singapore. The input will be provided to you in following format:'content: <content>\ncategory: <category>'. 
You need to match the category to a key in the dictionary below and return the most suitable subcategory from the corresponding list in the following format: 'category: <category>'. 
Do not make up any new categories. If there is no key in the dictionary that matches the input return'other'.\n{cross_reference}"""

for index, row in df.iterrows():
    category = row["category2"]
    content = row["summary"]
    input = f'content: {content}\ncategory: {category}'
    try:
        output = generate_gemini_content(input, prompt2, model)
    except:
        categories_3.append("couldn't generate")
    else:
        categories_3.append(output.split(':')[-1])

In [ ]:
df["category3"] = categories_3

In [ ]:
df.to_csv('data/reddit_data_w_summary_categories.csv')
df.to_json('data/reddit_data_w_summary_categories.json')

In [ ]:
df_obj = df.select_dtypes('object')
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

### Report Generation

In [ ]:
df["category1"].value_counts()

In [ ]:
df["category2"].value_counts()

In [ ]:
for cat in cross_reference:
    print(df.loc[df['category2'] == cat]["category3"].value_counts())

In [ ]:
import matplotlib.pyplot as plt
labels = df["category2"].value_counts().keys()
sizes = df["category2"].value_counts()
fig, ax = plt.subplots()
ax.pie(sizes, labels=labels)